CONTEXTE : Vous travaillez pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près à la consommation et aux émissions des bâtiments non destinés à l’habitation.

OBJECTIF : vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l’habitation pour lesquels elles n’ont pas encore été mesurées.

NB : Votre prédiction se basera sur les données structurelles des bâtiments (taille et usage des bâtiments, date de construction, situation géographique, ...)


TO DO :
En + : Vous cherchez également à évaluer l’intérêt de l’"ENERGY STAR Score" pour la prédiction d’émissions, qui est fastidieux à calculer avec l’approche utilisée actuellement par votre équipe. Vous l'intégrerez dans la modélisation et jugerez de son intérêt.

1) Réaliser une courte analyse exploratoire.
2) Tester différents modèles de prédiction afin de répondre au mieux à la problématique.


CONSEILS :
L’objectif est de te passer des relevés de consommation annuels futurs (attention à la fuite de données / DATA LEAKS). Nous ferons de toute façon pour tout nouveau bâtiment un premier relevé de référence la première année, donc rien ne t'interdit d’en déduire des variables structurelles aux bâtiments, par exemple la nature et proportions des sources d’énergie utilisées..

Fais bien attention au traitement des différentes variables, à la fois pour trouver de nouvelles informations (peut-on déduire des choses intéressantes d’une simple adresse ?) et optimiser les performances en appliquant des transformations simples aux variables (normalisation, passage au log, etc.).

Mets en place une évaluation rigoureuse des performances de la régression, et optimise les hyperparamètres et le choix d’algorithmes de ML à l’aide d’une validation croisée.

fuite des données ? :

In [42]:
%reset -f
%matplotlib inline
%autosave 300

import pandas as pd
import numpy as np
import seaborn as sns


Autosaving every 300 seconds


In [43]:
# load raw dataset
#all_data = pd.read_csv("dataset/2016_Building_Energy_Benchmarking.csv", sep = ',', encoding = 'utf-8')
all_data = pd.read_csv("dataset/2016_Building_Energy_Benchmarking.csv")

display(all_data)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.00000,0659000030,...,1156514.25000,3946027.00000,12764.52930,1276453.00000,False,NaN,Compliant,NaN,249.98000,2.83000
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.00000,0659000220,...,950425.18750,3242851.00000,51450.81641,5145082.00000,False,NaN,Compliant,NaN,295.86000,2.86000
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.00000,0659000475,...,14515435.00000,49526664.00000,14938.00000,1493800.00000,False,NaN,Compliant,NaN,2089.28000,2.19000
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.00000,0659000640,...,811525.31250,2768924.00000,18112.13086,1811213.00000,False,NaN,Compliant,NaN,286.43000,4.67000
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.00000,0659000970,...,1573448.62500,5368607.00000,88039.98438,8803998.00000,False,NaN,Compliant,NaN,505.01000,2.88000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,50222,2016,Nonresidential COS,Office,Horticulture building,1600 S Dakota St,Seattle,WA,NaN,1624049080,...,153655.00000,524270.86000,3254.75024,325475.02440,True,NaN,Error - Correct Default Data,NaN,20.94000,1.70000
3372,50223,2016,Nonresidential COS,Other,International district/Chinatown CC,719 8th Ave S,Seattle,WA,NaN,3558300000,...,116221.00000,396546.05200,5537.29980,553729.98050,False,NaN,Compliant,NaN,32.17000,2.01000
3373,50224,2016,Nonresidential COS,Other,Queen Anne Pool,1920 1st Ave W,Seattle,WA,NaN,1794501150,...,525251.68750,1792158.75800,39737.39063,3973739.06300,False,NaN,Compliant,NaN,223.54000,16.99000
3374,50225,2016,Nonresidential COS,Mixed Use Property,South Park Community Center,8319 8th Ave S,Seattle,WA,NaN,7883603155,...,102248.00000,348870.17600,3706.01001,370601.00100,False,NaN,Compliant,NaN,22.11000,1.57000


What's in this Dataset?
Rows : 3,376
Columns : 46
Each row is a : Building


source : https://data.seattle.gov/dataset/2016-Building-Energy-Benchmarking/2bpz-gwpy

In [44]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   PropertyName                     3376 non-null   object 
 5   Address                          3376 non-null   object 
 6   City                             3376 non-null   object 
 7   State                            3376 non-null   object 
 8   ZipCode                          3360 non-null   float64
 9   TaxParcelIdentificationNumber    3376 non-null   object 
 10  CouncilDistrictCode              3376 non-null   int64  
 11  Neighborhood                     3376 non-null   object 
 12  Latitude            

In [45]:
# Primary Key : OSEBuilding ID
all_data["OSEBuildingID"].unique().shape

(3376,)

In [46]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
all_data.describe()

,OSEBuildingID,DataYear,ZipCode,CouncilDistrictCode,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,...,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),Comments,TotalGHGEmissions,GHGEmissionsIntensity
count,3376.00000,3376.00000,3360.00000,3376.00000,3376.00000,3376.00000,3376.00000,3368.00000,3376.00000,3376.00000,...,3371.00000,3370.00000,3367.00000,3367.00000,3367.00000,3367.00000,3367.00000,0.00000,3367.00000,3367.00000
mean,21208.99111,2016.00000,98116.94911,4.43928,47.62403,-122.33480,1968.57316,1.10689,4.70912,94833.53732,...,5403667.29453,5276725.71439,274595.89821,1086638.96657,3707612.16159,13685.04538,1368504.54144,NaN,119.72397,1.17592
std,12223.75701,0.00000,18.61520,2.12063,0.04776,0.02720,33.08816,2.10840,5.49446,218837.60712,...,21610628.62764,15938786.48412,3912173.39270,4352478.35521,14850656.13896,67097.80830,6709780.83488,NaN,538.83223,1.82145
min,1.00000,2016.00000,98006.00000,1.00000,47.49917,-122.41425,1900.00000,0.00000,0.00000,11285.00000,...,0.00000,0.00000,0.00000,-33826.80078,-115417.00000,0.00000,0.00000,NaN,-0.80000,-0.02000
25%,19990.75000,2016.00000,98105.00000,3.00000,47.59986,-122.35066,1948.00000,1.00000,2.00000,28487.00000,...,925128.59375,970182.23438,0.00000,187422.94535,639487.00000,0.00000,0.00000,NaN,9.49500,0.21000
50%,23112.00000,2016.00000,98115.00000,4.00000,47.61867,-122.33249,1975.00000,1.00000,4.00000,44175.00000,...,1803753.25000,1904452.00000,0.00000,345129.90630,1177583.00000,3237.53760,323754.00000,NaN,33.92000,0.61000
75%,25994.25000,2016.00000,98122.00000,7.00000,47.65712,-122.31941,1997.00000,1.00000,5.00000,90992.00000,...,4222455.25000,4381429.12500,0.00000,829317.84375,2829632.50000,11890.33496,1189033.50000,NaN,93.94000,1.37000
max,50226.00000,2016.00000,98272.00000,7.00000,47.73387,-122.22097,2015.00000,111.00000,99.00000,9320156.00000,...,873923712.00000,471613856.00000,134943456.00000,192577488.00000,657074389.00000,2979090.00000,297909000.00000,NaN,16870.98000,34.09000


In [47]:
all_data["ListOfAllPropertyUseTypes"].unique().tolist()

['Hotel',
 'Hotel, Parking, Restaurant',
 'Hotel, Parking, Swimming Pool',
 'Police Station',
 'Other - Entertainment/Public Assembly',
 'Multifamily Housing',
 'Hotel, Parking',
 'Data Center, Library, Parking',
 'Fitness Center/Health Club/Gym, Office, Swimming Pool',
 'Office, Other, Other - Lodging/Residential, Restaurant, Social/Meeting Hall',
 'Courthouse',
 'Other',
 'K-12 School',
 'College/University',
 'Automobile Dealership',
 'Hotel, Parking, Retail Store, Swimming Pool',
 'K-12 School, Multifamily Housing, Office, Other - Entertainment/Public Assembly',
 'Self-Storage Facility',
 'Energy/Power Station, Laboratory, Manufacturing/Industrial Plant, Non-Refrigerated Warehouse, Office',
 'Non-Refrigerated Warehouse, Office, Refrigerated Warehouse',
 'Non-Refrigerated Warehouse, Office',
 'Office, Other - Education',
 'Office, Self-Storage Facility, Vocational School',
 'Data Center, Office, Retail Store',
 'Other - Mall, Personal Services (Health/Beauty, Dry Cleaning, etc)',
 '

In [48]:
all_data["BuildingType"].unique()

array(['NonResidential', 'Nonresidential COS', 'Multifamily MR (5-9)',
       'SPS-District K-12', 'Campus', 'Multifamily LR (1-4)',
       'Multifamily HR (10+)', 'Nonresidential WA'], dtype=object)

In [49]:
#all_data["PropertyName"].unique()
l = sorted(all_data["PrimaryPropertyType"].unique().tolist())
l

['Distribution Center',
 'High-Rise Multifamily',
 'Hospital',
 'Hotel',
 'K-12 School',
 'Laboratory',
 'Large Office',
 'Low-Rise Multifamily',
 'Medical Office',
 'Mid-Rise Multifamily',
 'Mixed Use Property',
 'Office',
 'Other',
 'Refrigerated Warehouse',
 'Residence Hall',
 'Restaurant',
 'Retail Store',
 'Self-Storage Facility',
 'Senior Care Community',
 'Small- and Mid-Sized Office',
 'Supermarket / Grocery Store',
 'University',
 'Warehouse',
 'Worship Facility']

In [50]:
np.sort(all_data["PrimaryPropertyType"].unique())

array(['Distribution Center', 'High-Rise Multifamily', 'Hospital',
       'Hotel', 'K-12 School', 'Laboratory', 'Large Office',
       'Low-Rise Multifamily', 'Medical Office', 'Mid-Rise Multifamily',
       'Mixed Use Property', 'Office', 'Other', 'Refrigerated Warehouse',
       'Residence Hall', 'Restaurant', 'Retail Store',
       'Self-Storage Facility', 'Senior Care Community',
       'Small- and Mid-Sized Office', 'Supermarket / Grocery Store',
       'University', 'Warehouse', 'Worship Facility'], dtype=object)

In [53]:
all_data["City"].unique()
all_data["State"].unique()
#all_data["ZipCode"].unique()
#all_data["CouncilDistrictCode"].unique() # 1 to 7

# Idea : councildistrictcode / neighborhood

a = all_data["Neighborhood"].unique().tolist()


b = [e.lower() for e in a]
b = set(b)
b

b = list(b)
len(b) # 14 après

len(a) # 19 avant:

19

19 unique neighborhoods, but because of upper case / lower case we get 14 neighborhoods.
We will use CouncilDistrictCode to localize the buildings.

In [54]:
columns_to_drop = ["PropertyName", "Address", "City", "State", "TaxParcelIdentificationNumber", "Neighborhood", "YearsENERGYSTARCertified", "DefaultData", "Comments", "ComplianceStatus"] ###

In [59]:
all_data_v1 = all_data.drop(columns=columns_to_drop)
display(all_data_v1)
# fillNaN for GFA
columns_to_fill = ["PropertyGFATotal", "PropertyGFAParking", "PropertyGFABuilding(s)", "ListOfAllPropertyUseTypes", "LargestPropertyUseType", "LargestPropertyUseTypeGFA", "SecondLargestPropertyUseType", "SecondLargestPropertyUseTypeGFA", "ThirdLargestPropertyUseType", "ThirdLargestPropertyUseTypeGFA", "Outlier"]

all_data_v1[columns_to_fill]

#GFA_outlier_example = all_data_v1.iloc[16]
GFA_outlier_example = all_data[all_data.index.isin([16])]
GFA_outlier_example




,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,ZipCode,CouncilDistrictCode,Latitude,Longitude,YearBuilt,NumberofBuildings,...,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,98101.00000,7,47.61220,-122.33799,1927,1.00000,...,7226362.50000,7456910.00000,2003882.00000,1156514.25000,3946027.00000,12764.52930,1276453.00000,NaN,249.98000,2.83000
1,2,2016,NonResidential,Hotel,98101.00000,7,47.61317,-122.33393,1996,1.00000,...,8387933.00000,8664479.00000,0.00000,950425.18750,3242851.00000,51450.81641,5145082.00000,NaN,295.86000,2.86000
2,3,2016,NonResidential,Hotel,98101.00000,7,47.61393,-122.33810,1969,1.00000,...,72587024.00000,73937112.00000,21566554.00000,14515435.00000,49526664.00000,14938.00000,1493800.00000,NaN,2089.28000,2.19000
3,5,2016,NonResidential,Hotel,98101.00000,7,47.61412,-122.33664,1926,1.00000,...,6794584.00000,6946800.50000,2214446.25000,811525.31250,2768924.00000,18112.13086,1811213.00000,NaN,286.43000,4.67000
4,8,2016,NonResidential,Hotel,98121.00000,7,47.61375,-122.34047,1980,1.00000,...,14172606.00000,14656503.00000,0.00000,1573448.62500,5368607.00000,88039.98438,8803998.00000,NaN,505.01000,2.88000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,50222,2016,Nonresidential COS,Office,NaN,2,47.56722,-122.31154,1990,1.00000,...,849745.68750,943003.18750,0.00000,153655.00000,524270.86000,3254.75024,325475.02440,NaN,20.94000,1.70000
3372,50223,2016,Nonresidential COS,Other,NaN,2,47.59625,-122.32283,2004,1.00000,...,950276.18750,1053705.75000,0.00000,116221.00000,396546.05200,5537.29980,553729.98050,NaN,32.17000,2.01000
3373,50224,2016,Nonresidential COS,Other,NaN,7,47.63644,-122.35784,1974,1.00000,...,5765898.00000,6053764.50000,0.00000,525251.68750,1792158.75800,39737.39063,3973739.06300,NaN,223.54000,16.99000
3374,50225,2016,Nonresidential COS,Mixed Use Property,NaN,1,47.52832,-122.32431,1989,1.00000,...,719471.18750,782841.31250,0.00000,102248.00000,348870.17600,3706.01001,370601.00100,NaN,22.11000,1.57000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   ZipCode                          3360 non-null   float64
 5   CouncilDistrictCode              3376 non-null   int64  
 6   Latitude                         3376 non-null   float64
 7   Longitude                        3376 non-null   float64
 8   YearBuilt                        3376 non-null   int64  
 9   NumberofBuildings                3368 non-null   float64
 10  NumberofFloors                   3376 non-null   int64  
 11  PropertyGFATotal                 3376 non-null   int64  
 12  PropertyGFAParking  

In [75]:
all_data_v1.info()
# ZipCode missing 16 $$$

l = all_data["ZipCode"].unique().tolist()

print(l)
zipcode_na = all_data[all_data["ZipCode"].isna()]
zipcode_na_list_address = zipcode_na["Address"].tolist()


zipcode_na_list = [[i, ""] for i in zipcode_na_list_address]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   ZipCode                          3360 non-null   float64
 5   CouncilDistrictCode              3376 non-null   int64  
 6   Latitude                         3376 non-null   float64
 7   Longitude                        3376 non-null   float64
 8   YearBuilt                        3376 non-null   int64  
 9   NumberofBuildings                3368 non-null   float64
 10  NumberofFloors                   3376 non-null   int64  
 11  PropertyGFATotal                 3376 non-null   int64  
 12  PropertyGFAParking  

In [77]:
zipcode_na_list

right_zipcode = [98125, 98144, 98117, 98125, 98107, 98117, 98119, 98112, 98122, 98118, 98126, 98108, 98104, 98119, 98108, 98108]

print(len(right_zipcode))

for i, zipcode in enumerate(right_zipcode):
    zipcode_na_list[i][1] = zipcode

print(zipcode_na_list)

16
[['10510 5th Ave NE', 98125], ['405 MLK Jr Way S', 98144], ['8729 15th Ave NW', 98117], ['2806 NE 127th St', 98125], ['6020 28th ave NW', 98107], ['1471 NW 67th St', 98117], ['1403 w howe', 98119], ['1400 E Galer St', 98112], ['2323 East Cherry St', 98122], ['4420 S Genesee', 98118], ['6920 34th Ave SW', 98126], ['1600 S Dakota St', 98108], ['719 8th Ave S', 98104], ['1920 1st Ave W', 98119], ['8319 8th Ave S', 98108], ['2820 S Myrtle St', 98108]]


In [73]:
l = [("a", "b")]

l[0][1]

'b'

In [56]:
def fill_second_third_property(data_frame, columns_to_fill):
    """

    :param data_frame:
    :param columns_to_fill:
    :return:
    """
    df = data_frame.copy()
    df = df
    pass

In [57]:
all_data_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   ZipCode                          3360 non-null   float64
 5   CouncilDistrictCode              3376 non-null   int64  
 6   Latitude                         3376 non-null   float64
 7   Longitude                        3376 non-null   float64
 8   YearBuilt                        3376 non-null   int64  
 9   NumberofBuildings                3368 non-null   float64
 10  NumberofFloors                   3376 non-null   int64  
 11  PropertyGFATotal                 3376 non-null   int64  
 12  PropertyGFAParking  

In [58]:
all_data[~all_data["Outlier"].isna()]

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
226,350,2016,NonResidential,Large Office,Second And Spring Building,1100 Second Avenue,Seattle,WA,98101.00000,0942000045,...,11663150.00000,39794668.00000,16048.30957,1604831.00000,False,NaN,Non-Compliant,High outlier,362.66000,2.10000
384,539,2016,NonResidential,Retail Store,University Center,4501 Roosevelt Way NE,Seattle,WA,98105.00000,5335200380,...,93482.60156,318963.00000,0.00000,0.00000,False,NaN,Non-Compliant,Low outlier,2.22000,0.03000
448,608,2016,NonResidential,Large Office,411 1ST AVE S (ID608),411 1ST AVE S,Seattle,WA,98104.00000,5247800200,...,471503.00000,1608768.00000,3430.50049,343050.00000,False,NaN,Non-Compliant,Low outlier,29.43000,0.19000
517,704,2016,NonResidential,Large Office,401 Elliott Ave West,401 Elliot Ave West,Seattle,WA,98119.00000,7666202110,...,7953454.50000,27137187.00000,0.00000,0.00000,False,NaN,Non-Compliant,High outlier,189.18000,1.46000
720,19708,2016,Multifamily HR (10+),High-Rise Multifamily,Bay Vista,2821 2nd Ave,Seattle,WA,98121.00000,0586400000,...,626561.62500,2137828.00000,2700.76001,270076.00000,False,NaN,Non-Compliant,Low outlier,29.25000,0.17000
834,19934,2016,Multifamily LR (1-4),Low-Rise Multifamily,Belmont Lofts,231 Belmont Ave E,Seattle,WA,98102.00000,0704600000,...,75020.89844,255971.00000,0.00000,0.00000,False,NaN,Non-Compliant,Low outlier,1.78000,0.06000
839,19967,2016,SPS-District K-12,K-12 School,Queen Anne Gym,1431 2nd Ave N,Seattle,WA,98109.00000,0811000005,...,72282.39844,246628.00000,1848.43958,184844.00000,True,NaN,Error - Correct Default Data,Low outlier,11.54000,0.32000
962,20367,2016,Multifamily LR (1-4),Low-Rise Multifamily,City Lights on Harbor,2349 Harbor Ave. SW,Seattle,WA,98126.00000,1595200000,...,205885.50000,702481.00000,0.00000,0.00000,False,NaN,Non-Compliant,Low outlier,4.90000,0.07000
1045,20617,2016,NonResidential,Retail Store,Seattle Habitat Store,21 S Nevada St,Seattle,WA,98134.00000,1824049074,...,14362.50000,49005.00000,964.64001,96464.00000,True,NaN,Error - Correct Default Data,Low outlier,5.46000,0.14000
1229,21315,2016,NonResidential,Small- and Mid-Sized Office,1518 Fifith Ave,1520 Fifth Ave,Seattle,WA,98101.00000,1975700135,...,7064.89990,24105.00000,0.00000,0.00000,False,NaN,Non-Compliant,Low outlier,0.17000,0.00000
